In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
use = pd.read_csv('eurostat/Water_Use_Data_RBD.csv', )
plant = pd.read_csv('eurostat/Treatment_plants_by_type_of_treatment_and_RBD.csv')
abstr = pd.read_csv('eurostat/Water_Abstraction_Data_RBD.csv')
waste = pd.read_csv('eurostat/Wastewater_generation_and_discharge_by_RBD.csv')
treat = pd.read_csv('eurostat/Treatment_capacity_of_wastewater_treatment_plants_by_RBD.csv')
pop = pd.read_csv('eurostat/Population_connected_to_wastewater_collection_and_treatment_systems_by_RBD.csv')
fresh = pd.read_csv('eurostat/Freshwater_Resources_Data.csv')

In [3]:
#fD = f[f['RBD'].str.contains("DE")]
#set(fD.WAT_PROC)
# years = np.arange(2000,2016)
# rbd = 'DE'
vals = ['use','plant','abstr','waste','treat','pop','fresh']
var = [use, plant, abstr, waste, treat, pop, fresh]
valsmust = ['fresh','waste','plant']
# for item in var:
#     print(item.size)
    
for i in vals:
    
    v = vars()[i]
    v[v.Value == ':'] = np.nan
#    print(v.Value.head())
    v.drop('Flag and Footnotes',axis=1,inplace=True)
    v.dropna(subset=['Value'],inplace=True)#axis=0, how='any',)
    v.set_index(['RBD'],inplace=True)#,'TIME']) 
   # print(v.columns.values)
    #v = v[v.TIME == 2001]
   # v.drop('TIME',axis=1,inplace=True)
    
    vars()[i] = v

# print('\n')
# for item in var:
#     print(item.size)


In [4]:
cols = [['NACE_R2','WAT_PROC'],
        ['WW_TP','WW_TPAR'],
        ['WAT_PROC','WAT_SRC'],
        ['WW_GTD','SUBSTANCE'],
        ['WW_TPAR','WW_TP'],
        ['WW_TP'],
        ['WAT_PROC'],]

In [5]:
use = use.set_index(['NACE_R2','WAT_PROC','UNIT'],append=True)
plant = plant.set_index(['WW_TP','UNIT','WW_TPAR'],append=True)
abstr = abstr.set_index(['WAT_PROC','WAT_SRC','UNIT'],append=True)
waste = waste.set_index(['WW_GTD','SUBSTANCE','UNIT'],append=True)
treat = treat.set_index(['WW_TPAR','WW_TP','UNIT'],append=True)
pop = pop.set_index(['UNIT','WW_TP'],append=True)
fresh = fresh.set_index(['UNIT','WAT_PROC'],append=True)

In [6]:
l = set(use.index.get_level_values('RBD').values)
valsmust = ['fresh','waste','plant','pop']#,'abstr']
for v in valsmust[1:]:
    l = set(vars()[v].index.get_level_values('RBD').values) & l


In [11]:
for v in vals:
    x = vars()[v].index.get_level_values('RBD').str.contains('Danube')
    vars()[v] = vars()[v][x]
    if vars()[v].size == 0:
        print('{} is empty.'.format(v))

use [ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  Tru

In [8]:
p = pd.Series()
d = pd.Series()
sub = pd.Series()
for i, v in enumerate(vals):
    for idx in cols[i]:
        x = list(set(vars()[v].index.get_level_values(idx)))
        sub = sub.append(pd.Series([idx]*len(x)),
                ignore_index=True)
        p = p.append(pd.Series(x),
                ignore_index=True)
        d = d.append(pd.Series([v]*len(x)),
                    ignore_index=True)
params = pd.DataFrame({'DataSet':d,'DataSetColumn':sub,'Parameter':p})
params.tail(4)

,DataSet,DataSetColumn,Parameter
49,pop,WW_TP,Urban and other wastewater treatment plants - ...
50,pop,WW_TP,Resident population
51,pop,WW_TP,Urban wastewater collecting system
52,fresh,WAT_PROC,Precipitation


In [72]:
l = []
for i, v in enumerate(vals):
    l.append(np.asarray(list(set(vars()[v].index.get_level_values('RBD').values))))
danube = list(set([i for item in l for i in item]))
rem = [item for item in danube if ('RBD' not in item)]
for i in rem:
    danube.remove(i) 

ds = [item[-3:-1] for item in danube]
danube

['Danube - RBD (CH)',
 'Danube - RBD (DE)',
 'Danube - RBD (HU)',
 'Danube - RBD (SI)',
 'Danube - RBD (AT)',
 'Danube - RBD (CZ)',
 'Danube - RBD (PL)',
 'Danube - RBD (SK)',
 'Danube - RBD (RO)',
 'Danube - RBD (BG)',
 'Danube - RBD (HR)']

In [73]:
header = params.Parameter.tolist()
for i, dan in enumerate(danube):
    print(dan)
    db = pd.DataFrame(index=np.arange(2000,2016))
    for i, row in params.iterrows():
        df = vars()[row.DataSet]
        subdata = df[(df.index.get_level_values(row.DataSetColumn)==row.Parameter) & (df.index.get_level_values('RBD')==dan)]
        units = list(set(subdata.index.get_level_values('UNIT').values))
        if units != []:
            for unit in units:
                ssub = subdata[subdata.index.get_level_values('UNIT') == unit]
                time = ssub.TIME.tolist()
                values = ssub.Value.astype(float).tolist()
                s = pd.DataFrame({'time':time,'values':values})
                s = s.groupby('time').mean()
                db['{} ({})'.format(row.Parameter,unit)] = s
    db.to_csv('Danube/{}.csv'.format(dan[-3:-1]))

Danube - RBD (CH)
Danube - RBD (DE)
Danube - RBD (HU)
Danube - RBD (SI)
Danube - RBD (AT)
Danube - RBD (CZ)
Danube - RBD (PL)
Danube - RBD (SK)
Danube - RBD (RO)
Danube - RBD (BG)
Danube - RBD (HR)


In [62]:
sk1 =  pd.read_csv('Danube/SK.csv')
sk2 = pd.read_csv('Danube/SK_2.csv')


In [63]:
sk1

,Unnamed: 0,Water abstraction for agriculture - irrigation (Million cubic metres),Water abstraction for public water supply (Million cubic metres),Total gross abstraction (Million cubic metres),Desalinated water (Million cubic metres),Fresh groundwater (Million cubic metres),Fresh surface water (Million cubic metres),"Generation of wastewater - agriculture, forestry, fishing (Tonnes of O2 per day)","Generation of wastewater - agriculture, forestry, fishing (Million cubic metres)","Generation of wastewater - agriculture, forestry, fishing (Tonnes per day)",...,Generation of wastewater - point sources - total (Tonnes of O2 per day),Generation of wastewater - point sources - total (Million cubic metres),Generation of wastewater - point sources - total (Tonnes per day),Generation of wastewater - manufacturing industries (Tonnes of O2 per day),Generation of wastewater - manufacturing industries (Million cubic metres),Generation of wastewater - manufacturing industries (Tonnes per day),Nitrogen (N-tot ) (Tonnes per day),Biochemical oxygen demand (BOD) (Tonnes of O2 per day),Volume (Million cubic metres),Precipitation (Million cubic metres)
0,2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2004,19.710,173.940,337.746667,0.0,336.605000,235.873333,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2005,11.010,167.615,297.803333,0.0,326.620000,195.470000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2006,15.850,163.755,250.250000,0.0,318.575000,152.320000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2007,9.420,157.420,226.046667,0.0,308.325000,128.583333,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2008,9.040,156.955,218.316667,0.0,304.900000,122.700000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2009,12.320,155.710,206.306667,0.0,302.815000,112.343333,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
sk2

,Unnamed: 0,Water abstraction for agriculture - irrigation (Million cubic metres),Water abstraction for public water supply (Million cubic metres),Total gross abstraction (Million cubic metres),Desalinated water (Million cubic metres),Fresh groundwater (Million cubic metres),Fresh surface water (Million cubic metres),"Generation of wastewater - agriculture, forestry, fishing (Tonnes of O2 per day)","Generation of wastewater - agriculture, forestry, fishing (Million cubic metres)","Generation of wastewater - agriculture, forestry, fishing (Tonnes per day)",...,Generation of wastewater - services (Tonnes per day),Generation of wastewater - point sources - total (Tonnes of O2 per day),Generation of wastewater - point sources - total (Million cubic metres),Generation of wastewater - point sources - total (Tonnes per day),Generation of wastewater - manufacturing industries (Tonnes of O2 per day),Generation of wastewater - manufacturing industries (Million cubic metres),Generation of wastewater - manufacturing industries (Tonnes per day),Nitrogen (N-tot ) (Tonnes per day),Biochemical oxygen demand (BOD) (Tonnes of O2 per day),Volume (Million cubic metres)
0,2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2004,1.960,46.610,53.230000,0.0,96.555000,20.586667,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2005,1.080,28.320,35.746667,0.0,58.980000,15.666667,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2006,0.860,29.375,37.720000,0.0,60.975000,16.940000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2007,0.160,30.860,41.120000,0.0,64.170000,18.966667,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2008,1.120,30.580,51.490000,0.0,77.975000,20.266667,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2009,1.510,30.740,49.686667,0.0,77.835000,18.793333,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
for i, row in params.iterrows():
    df = vars()[row.DataSet]
    subdata = df[df.index.get_level_values(row.DataSetColumn)==row.Parameter]
    u = set(subdata.index.get_level_values('UNIT'))
    if len(u) != 1:
        print('{} has multiple units'.format(row.Parameter))
        unit = 0
        continue
    elif len(u) == 1:
        unit = list(u)[0]
#     plt.figure(figsize=(7,5))
#     ax = plt.subplot(111)
    for region in list(set(df.index.get_level_values('RBD').values)):
        subsubdata = subdata[subdata.index.get_level_values('RBD')==region]
subsubdata
        #subsubdata.TIME = subsubdata.TIME.astype(float) 
        #print(subsubdata.head())#.groupby(by = 'TIME').mean())
#         time = subsubdata.TIME
#         values = subsubdata.Value
#         plt.scatter(time, values,label='{}'.format(region))
#     plt.title('{} in {}'.format(row.Parameter, unit))
#     ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

Generation of wastewater - agriculture, forestry, fishing has multiple units
Generation of wastewater - industry total has multiple units
Total wastewater discharged - without treatment has multiple units
Total discharges of wastewater treatment plants (urban and other) has multiple units
Generation of wastewater - construction has multiple units
Generation of wastewater - private households has multiple units
Generation of wastewater - mining and quarrying has multiple units
Total discharges to inland waters has multiple units
Generation of wastewater - services has multiple units
Urban and industrial wastewater treatment - total inflow has multiple units
Generation of wastewater - point sources - total has multiple units
Generation of wastewater - manufacturing industries has multiple units


TIME  Value
RBD               UNIT                 WAT_PROC                    
Danube - RBD (BG) Million cubic metres Precipitation  2014.0  45026
                                       Precipitation  2015.0  31038

In [240]:

vals2 = vals[1:]
for i, v in enumerate(valsmust):
    print(vars()[v].head())
   # df = df.join(vars()[v],how='inner',rsuffix='_{}'.format(v)).dropna(axis=1, how='all')

Empty DataFrame
Columns: [Value]
Index: []
                                                                                                      Value
RBD               WW_GTD                                             SUBSTANCE UNIT                        
Danube - RBD (BG) Generation of wastewater - point sources - total   Volume    Million cubic metres  240.72
                  Generation of wastewater - agriculture, forestr... Volume    Million cubic metres    2.34
                  Generation of wastewater - industry total          Volume    Million cubic metres   98.82
                  Generation of wastewater - mining and quarrying    Volume    Million cubic metres    7.13
                  Generation of wastewater - manufacturing indust... Volume    Million cubic metres   73.07
                                                                                             Value
RBD               WW_TP                                              UNIT   WW_TPAR               
Dan

In [117]:
df.size

98996235

In [102]:
f07.Value = f07.Value.astype(float)
f07 = pd.get_dummies(f07)
d07.Value = d07.Value.astype(float)
d07 = pd.get_dummies(d07)
#d07.columns.values
print(d07.columns.values)
for col in d07.columns.values:
    if col != 'Value':
        print(col, d07[col][d07[col]!=1].values)
        if d07[col][d07[col]!=1].values.size == 0:
            d07 = d07.drop(columns=col,axis=1)
print(d07.columns.values)
d07 = d07.rename(index=str, columns={'Value':'val_{}'.format()})

['Value' 'UNIT_Number' 'WW_TPAR_Number of plants'
 'WW_TP_Urban and other wastewater treatment plants - primary treatment'
 'WW_TP_Urban and other wastewater treatment plants - secondary treatment'
 'WW_TP_Urban and other wastewater treatment plants - tertiary treatment']
UNIT_Number []
WW_TPAR_Number of plants []
WW_TP_Urban and other wastewater treatment plants - primary treatment [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
WW_TP_Urban and other wastewater treatment plants - secondary treatment [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
WW_TP_Urban and other wastewater treatment plants - tertiary treatment [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
['Value'
 'WW_TP_Urban and other wastewater treatment plants - primary treatment'
 'WW_TP_Urban and other wastewater treatment plants - secondary treatment'
 'WW_TP_Urban and other wastewater treatment plants - tertiary treatment']


Value  \
RBD                     TIME            
Danube - RBD (DE)       2007   337.00   
                        2007  1006.07   
                        2007   869.01   
Rhine - RBD (DE)        2007  1190.00   
                        2007  1548.00   
                        2007  2184.00   
Moselle-Saar (DE)       2007    58.00   
                        2007   248.85   
                        2007   229.00   
Ems - RBD (DE)          2007    81.00   
                        2007    40.00   
                        2007   204.00   
Elbe / Labe - RBD (DE)  2007   506.00   
                        2007  1350.94   
                        2007   942.00   
Oder / Odra - RBD (DE)  2007    10.00   
                        2007    83.00   
                        2007    68.00   
Maas / Meuse - RBD (DE) 2007    52.00   
                        2007    12.00   
                        2007    76.11   

                              WW_TP_Urban and other wastewater treatment plants - primary treatment  \
RBD                     TIME                                                                          
Danube - RBD (DE)       2007                                                  1                       
                        2007                                                  0                       
                        2007                                                  0                       
Rhine - RBD (DE)        2007                                                  1                       
                        2007                                                  0                       
                        2007                                                  0                       
Moselle-Saar (DE)       2007                                                  1                       
                        2007                                                  0                       
                        2007                                                  0                       
Ems - RBD (DE)          2007                                                  1                       
                        2007                                                  0                       
                        2007                                                  0                       
Elbe / Labe - RBD (DE)  2007                                                  1                       
                        2007                                                  0                       
                        2007                                                  0                       
Oder / Odra - RBD (DE)  2007                                                  1                       
                        2007                                                  0                       
                        2007                                                  0                       
Maas / Meuse - RBD (DE) 2007                                                  1                       
                        2007                                                  0                       
                        2007                                                  0                       

                              WW_TP_Urban and other wastewater treatment plants - secondary treatment  \
RBD                     TIME                                                                            
Danube - RBD (DE)       2007                                                  0                         
                        2007                                                  1                         
                        2007                                                  0                         
Rhine - RBD (DE)        2007                                                  0                         
                        2007                                                  1         

In [10]:
fD = fD[fD.Value != ':']
fD.Value = fD.Value.astype(float)

In [11]:

for rbd in fD.RBD.values[0]:
    df = fD[fD.RBD == rbd]
    for type_ in df.WAT_PROC.values:
        ddf = df[df.WAT_PROC.values == type_]
        #print(ddf[['TIME','Value']].describe())
        ddf.plot(x='TIME',y='Value')
        plt.title('Region: {}, {}'.format(rbd,type_))
        

In [ ]:
precip.plot(x='TIME',y='')